<br></br>

<div align="center">

<h1 align="center">
    Cuaderno para el trabajo de clasificación relacional
    <br></br>
    Evaluar modelos
</h1>

<h6 align="center">
    Antonio Macías Ferrera (antmacfer1@alum.us.es)
    <br></br>
    Delfín Santana Rubio (delsanrub@alum.us.es)
    <br></br>
    Universidad de Sevilla
</h6>

<br></br>

## **Control de Versiones**
    
| **Fecha**  | **Versión** | **Descripción**               |
| :--------- | :---------- | :---------------------------- |
| 27/05/2024 | v1r0        | Primera versión del cuaderno. |
| 29/05/2024 | v1r1        | Inclsuión de otras medidas de centralidad.          |
| 31/05/2024 | v1r2        | Correcciones a algunos métodos e inclusión de otros.|
| 01/06/2024 | v1r2        | Eliminación de métricas no óptimas.                 |
| 07/06/2024 | v1r3        | Correcciones de formato.      |


</div>

<br></br>

## **Índice de contenido**

1. [Introducción](#introducción)
2. [Instanciación del grafo](#creacion)
3. [Evaluación de las métricas](#evaluacion)
    - [Árboles de decisión CART](#cart)
    - [Naive Bayes](#naive-bayes)
    - [KNN](#knn)
    - [Random Forest](#random-forest)
    - [Gradient Boosting](#gradient-boosting)

<br></br>

# <a name="introducción"></a> 1. **Introducción**

En este cuaderno se encuentra todo el codigo neceario para evaluar los modelos de aprendizaje automático con las distintas métricas seleccionadas anteriormente.

A continuación se importan todas las librerías y métodos necesarios para la ejecución del código.

<br></br>

In [1]:
# Pandas
import pandas as pd

# Numpy
import numpy as np

# Codificadores de sci-kit learn
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# Train-test split, búsqueda en rejilla, validaciones cruzadas y selección de modelos
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score

# Modelos a entrenar (CART, Naive Bayes, Knn, Support Vector Machines)
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Discretizador para Naive Bayes
from sklearn.preprocessing import KBinsDiscretizer

# Normalizador para Knn
from sklearn.preprocessing import MinMaxScaler

# Tubería
from sklearn.pipeline import Pipeline

# <a name="creacion"></a> 2. **Instanciación del grafo con sólo comunidades**

Aquí se cargan los datos del csv en un dataframe de Pandas, y se determinan calcula el accuracy de los modelos con sólo con las medidas relacionadas con la detección de comunidades.
<br></br>

In [2]:
values = pd.read_csv('dict.csv')
values.head()

,greedy_modularity_communities_id,label_propagation_communities_id,betweenness_centrality,degree_centrality,k_path_centrality,closeness_centrality,harmonic_centrality,square_clustering,clustering,target
0,5,0,0.000000,0.000045,0.000003,0.182482,4224.448882,0.000000,0.000000,tvshow
1,2,1,0.000153,0.001513,0.000100,0.257752,6230.390079,0.130612,0.481283,government
2,4,5,0.000003,0.000534,0.000035,0.189540,4419.497403,0.347089,0.651515,company
3,22,6,0.000052,0.000445,0.000029,0.219007,5147.241270,0.160836,0.511111,government
4,3,8,0.000125,0.002270,0.000149,0.230387,5493.355556,0.176843,0.420392,politician


In [3]:
atributos_continuos = ["greedy_modularity_communities_id", "label_propagation_communities_id"]#, "betweenness_centrality", "degree_centrality", "k_path_centrality", "closeness_centrality", "harmonic_centrality", "clustering", "square_clustering"]
atributos = values.loc[:, atributos_continuos]
atributos.head()

,greedy_modularity_communities_id,label_propagation_communities_id
0,5,0
1,2,1
2,4,5
3,22,6
4,3,8


In [4]:
objetivo = values['target']
objetivo.head()

0        tvshow
1    government
2       company
3    government
4    politician
Name: target, dtype: object

In [5]:
codificador_objetivo = LabelEncoder()
# El método fit_transform ajusta el codificador a los datos y, a continuación,
# codifica estos adecuadamente. En este caso no necesitamos mantener el
# atributo objetivo como una Series de Pandas.
objetivo = codificador_objetivo.fit_transform(objetivo)
print(f'Clases detectadas: {codificador_objetivo.classes_}')
objetivo

Clases detectadas: ['company' 'government' 'politician' 'tvshow']


array([3, 1, 0, ..., 1, 0, 3])

## <a name="cart"></a> **Árboles de decisión CART**

In [6]:
# Establecer división entre atributos de prueba y de entrenamiento 
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos, objetivo, 
    test_size=.2, 
    stratify=objetivo)

In [7]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, árbol de decisión CART)
rejilla = {
    'max_depth': [3, 5, 8, 12, None],
    'min_samples_split': [2, 10, 40, 50, 100, 200],
    'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
    'criterion': ['gini', 'entropy']
}

modelo = DecisionTreeClassifier()

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(estimator=modelo, param_grid=rejilla, cv=5, scoring='accuracy', n_jobs=-1)

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 8, 12, None],
                         'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
                         'min_samples_split': [2, 10, 40, 50, 100, 200]},
             scoring='accuracy')

In [8]:
# Mejor modelo obtenido
mejor_modelo_CART = validaciones_cruzadas.best_estimator_
mejor_modelo_CART

DecisionTreeClassifier(criterion='entropy')

In [9]:
validaciones_cruzadas.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [10]:
# Evaluar la importancia de los atributos
importancias_CART = mejor_modelo_CART.feature_importances_

# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_CART.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"- Accuracy: {accuracy}")

- Accuracy: 0.8466844681797953


## <a name="naive-bayes"></a> **Naive Bayes**

In [11]:
# Discretizar los atributos para poder usar Naive Bayes
discretizador = KBinsDiscretizer(
    n_bins=4,  # Cada atributo se discretiza en 4 intervalos
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='quantile'  # Cada intervalo contiene la misma cantidad de datos
)

atributos_discretizados = atributos.copy()
atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)

/opt/homebrew/lib/python3.12/site-packages/sklearn/preprocessing/_discretization.py:322: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


In [12]:
# Establecer división entre atributos de prueba y de entrenamiento 
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos_discretizados, objetivo, 
    test_size=.2, 
    stratify=objetivo)

In [13]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
rejilla = {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]}

modelo = CategoricalNB()

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(estimator=modelo, param_grid=rejilla, cv=10, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=10, estimator=CategoricalNB(),
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]},
             scoring='accuracy')

In [14]:
# Mejor modelo obtenido
mejor_modelo_NB = validaciones_cruzadas.best_estimator_
mejor_modelo_NB

CategoricalNB(alpha=0.1)

In [15]:
validaciones_cruzadas.best_params_

{'alpha': 0.1}

In [16]:
# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_NB.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"- Accuracy: {accuracy}")

- Accuracy: 0.43880729862038276


## <a name="knn"></a> **KNN**

In [17]:
# Establecer división entre atributos de prueba y de entrenamiento 
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos_discretizados, objetivo, 
    test_size=.2, 
    stratify=objetivo)

In [18]:
# Normalizar atributos
normalizador = MinMaxScaler(feature_range=(0,1))

# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
tuberia = Pipeline([('normalizador', normalizador),
                   ('knn', KNeighborsClassifier())])

rejilla = {
    'knn__n_neighbors': [1,3,5,7],
    'knn__metric': ['euclidean', 'manhattan']
}

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(tuberia, rejilla, cv=5, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('normalizador', MinMaxScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': [1, 3, 5, 7]},
             scoring='accuracy')

In [19]:
# Mejor modelo obtenido
mejor_modelo_KNN = validaciones_cruzadas.best_estimator_
mejor_modelo_KNN

Pipeline(steps=[('normalizador', MinMaxScaler()),
                ('knn',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=7))])

In [20]:
validaciones_cruzadas.best_params_

{'knn__metric': 'euclidean', 'knn__n_neighbors': 7}

In [21]:
# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_KNN.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"- Accuracy: {accuracy}")


- Accuracy: 0.3991989319092123


## <a name="random-forest"></a> **Random forest**

In [22]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
modelo = RandomForestClassifier()

rejilla = {
    'max_depth': [3, 5, 8, 12, None],
    'min_samples_split': [2, 10, 40, 50, 100, 200],
    'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
    'criterion': ['gini', 'entropy']
}

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(modelo, rejilla, cv=5, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 8, 12, None],
                         'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
                         'min_samples_split': [2, 10, 40, 50, 100, 200]},
             scoring='accuracy')

In [23]:
# Mejor modelo obtenido
mejor_modelo_RFC = validaciones_cruzadas.best_estimator_
mejor_modelo_RFC

RandomForestClassifier(max_depth=5)

In [24]:
validaciones_cruzadas.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [25]:
# Evaluar la importancia de los atributos
importancias_RFC = mejor_modelo_RFC.feature_importances_

# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_RFC.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"Accuracy: {accuracy}")

Accuracy: 0.4652870493991989


## <a name="gradient-boosting"></a> **Gradient Boosting**

In [26]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
modelo = GradientBoostingClassifier(random_state=42)

rejilla = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [1, 3, 5],
    'subsample': [0.8, 1.0]
}


# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(modelo, rejilla, cv=5, n_jobs=-1, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 1.0],
                         'max_depth': [1, 3, 5], 'n_estimators': [50, 100, 150],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy')

In [27]:
# Mejor modelo obtenido
mejor_modelo_GB = validaciones_cruzadas.best_estimator_
mejor_modelo_GB

GradientBoostingClassifier(n_estimators=50, random_state=42, subsample=0.8)

In [28]:
validaciones_cruzadas.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}

In [29]:
# Evaluar la importancia de los atributos
importancias_GB = mejor_modelo_GB.feature_importances_

# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_GB.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"Accuracy: {accuracy}")


Accuracy: 0.4652870493991989


# <a name="creacion"></a> 3. **Instanciación del grafo sin comunidades**

Aquí se cargan los datos del csv en un dataframe de Pandas, y se determinan calcula el accuracy de los modelos sin las medidas relacionadas con la detección de comunidades.
<br></br>

In [30]:
values = pd.read_csv('dict.csv')
values.head()

,greedy_modularity_communities_id,label_propagation_communities_id,betweenness_centrality,degree_centrality,k_path_centrality,closeness_centrality,harmonic_centrality,square_clustering,clustering,target
0,5,0,0.000000,0.000045,0.000003,0.182482,4224.448882,0.000000,0.000000,tvshow
1,2,1,0.000153,0.001513,0.000100,0.257752,6230.390079,0.130612,0.481283,government
2,4,5,0.000003,0.000534,0.000035,0.189540,4419.497403,0.347089,0.651515,company
3,22,6,0.000052,0.000445,0.000029,0.219007,5147.241270,0.160836,0.511111,government
4,3,8,0.000125,0.002270,0.000149,0.230387,5493.355556,0.176843,0.420392,politician


In [31]:
atributos_continuos = ["betweenness_centrality", "degree_centrality", "k_path_centrality", "closeness_centrality", "harmonic_centrality", "clustering", "square_clustering"]
atributos = values.loc[:, atributos_continuos]
atributos.head()

,betweenness_centrality,degree_centrality,k_path_centrality,closeness_centrality,harmonic_centrality,clustering,square_clustering
0,0.000000,0.000045,0.000003,0.182482,4224.448882,0.000000,0.000000
1,0.000153,0.001513,0.000100,0.257752,6230.390079,0.481283,0.130612
2,0.000003,0.000534,0.000035,0.189540,4419.497403,0.651515,0.347089
3,0.000052,0.000445,0.000029,0.219007,5147.241270,0.511111,0.160836
4,0.000125,0.002270,0.000149,0.230387,5493.355556,0.420392,0.176843


In [32]:
objetivo = values['target']
objetivo.head()

0        tvshow
1    government
2       company
3    government
4    politician
Name: target, dtype: object

In [33]:
codificador_objetivo = LabelEncoder()
# El método fit_transform ajusta el codificador a los datos y, a continuación,
# codifica estos adecuadamente. En este caso no necesitamos mantener el
# atributo objetivo como una Series de Pandas.
objetivo = codificador_objetivo.fit_transform(objetivo)
print(f'Clases detectadas: {codificador_objetivo.classes_}')
objetivo

Clases detectadas: ['company' 'government' 'politician' 'tvshow']


array([3, 1, 0, ..., 1, 0, 3])

## <a name="cart"></a> **Árboles de decisión CART**

In [34]:
# Establecer división entre atributos de prueba y de entrenamiento 
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos, objetivo, 
    test_size=.2, 
    stratify=objetivo)

In [35]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, árbol de decisión CART)
rejilla = {
    'max_depth': [3, 5, 8, 12, None],
    'min_samples_split': [2, 10, 40, 50, 100, 200],
    'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
    'criterion': ['gini', 'entropy']
}

modelo = DecisionTreeClassifier()

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(estimator=modelo, param_grid=rejilla, cv=5, scoring='accuracy', n_jobs=-1)

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 8, 12, None],
                         'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
                         'min_samples_split': [2, 10, 40, 50, 100, 200]},
             scoring='accuracy')

In [36]:
# Mejor modelo obtenido
mejor_modelo_CART = validaciones_cruzadas.best_estimator_
mejor_modelo_CART

DecisionTreeClassifier(criterion='entropy', min_samples_leaf=7,
                       min_samples_split=40)

In [37]:
validaciones_cruzadas.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'min_samples_leaf': 7,
 'min_samples_split': 40}

In [38]:
# Evaluar la importancia de los atributos
importancias_CART = mejor_modelo_CART.feature_importances_

# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_CART.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"- Accuracy: {accuracy}")

- Accuracy: 0.5520694259012016


## <a name="naive-bayes"></a> **Naive Bayes**

In [39]:
# Discretizar los atributos para poder usar Naive Bayes
discretizador = KBinsDiscretizer(
    n_bins=4,  # Cada atributo se discretiza en 4 intervalos
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='quantile'  # Cada intervalo contiene la misma cantidad de datos
)

atributos_discretizados = atributos.copy()
atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)

In [40]:
# Establecer división entre atributos de prueba y de entrenamiento 
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos_discretizados, objetivo, 
    test_size=.2, 
    stratify=objetivo)

In [41]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
rejilla = {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]}

modelo = CategoricalNB()

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(estimator=modelo, param_grid=rejilla, cv=10, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=10, estimator=CategoricalNB(),
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]},
             scoring='accuracy')

In [42]:
# Mejor modelo obtenido
mejor_modelo_NB = validaciones_cruzadas.best_estimator_
mejor_modelo_NB

CategoricalNB(alpha=2.0)

In [43]:
validaciones_cruzadas.best_params_

{'alpha': 2.0}

In [44]:
# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_NB.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"- Accuracy: {accuracy}")

- Accuracy: 0.4494882064975523


## <a name="knn"></a> **KNN**

In [45]:
# Establecer división entre atributos de prueba y de entrenamiento 
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos_discretizados, objetivo, 
    test_size=.2, 
    stratify=objetivo)

In [46]:
# Normalizar atributos
normalizador = MinMaxScaler(feature_range=(0,1))

# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
tuberia = Pipeline([('normalizador', normalizador),
                   ('knn', KNeighborsClassifier())])

rejilla = {
    'knn__n_neighbors': [1,3,5,7],
    'knn__metric': ['euclidean', 'manhattan']
}

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(tuberia, rejilla, cv=5, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('normalizador', MinMaxScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': [1, 3, 5, 7]},
             scoring='accuracy')

In [47]:
# Mejor modelo obtenido
mejor_modelo_KNN = validaciones_cruzadas.best_estimator_
mejor_modelo_KNN

Pipeline(steps=[('normalizador', MinMaxScaler()),
                ('knn',
                 KNeighborsClassifier(metric='euclidean', n_neighbors=7))])

In [48]:
validaciones_cruzadas.best_params_

{'knn__metric': 'euclidean', 'knn__n_neighbors': 7}

In [49]:
# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_KNN.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"- Accuracy: {accuracy}")


- Accuracy: 0.4652870493991989


## <a name="random-forest"></a> **Random forest**

In [50]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
modelo = RandomForestClassifier()

rejilla = {
    'max_depth': [3, 5, 8, 12, None],
    'min_samples_split': [2, 10, 40, 50, 100, 200],
    'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
    'criterion': ['gini', 'entropy']
}

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(modelo, rejilla, cv=5, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 8, 12, None],
                         'min_samples_leaf': [1, 2, 5, 7, 10, 20, 40],
                         'min_samples_split': [2, 10, 40, 50, 100, 200]},
             scoring='accuracy')

In [51]:
# Mejor modelo obtenido
mejor_modelo_RFC = validaciones_cruzadas.best_estimator_
mejor_modelo_RFC

RandomForestClassifier(min_samples_leaf=2, min_samples_split=100)

In [52]:
validaciones_cruzadas.best_params_

{'criterion': 'gini',
 'max_depth': None,
 'min_samples_leaf': 2,
 'min_samples_split': 100}

In [53]:
# Evaluar la importancia de los atributos
importancias_RFC = mejor_modelo_RFC.feature_importances_

# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_RFC.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5235870048954161


## <a name="gradient-boosting"></a> **Gradient Boosting**

In [54]:
# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
modelo = GradientBoostingClassifier(random_state=42)

rejilla = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [1, 3, 5],
    'subsample': [0.8, 1.0]
}


# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(modelo, rejilla, cv=5, n_jobs=-1, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 1.0],
                         'max_depth': [1, 3, 5], 'n_estimators': [50, 100, 150],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy')

In [55]:
# Mejor modelo obtenido
mejor_modelo_GB = validaciones_cruzadas.best_estimator_
mejor_modelo_GB

GradientBoostingClassifier(n_estimators=150, random_state=42)

In [56]:
validaciones_cruzadas.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 150, 'subsample': 1.0}

In [57]:
# Evaluar la importancia de los atributos
importancias_GB = mejor_modelo_GB.feature_importances_

# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_GB.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"Accuracy: {accuracy}")


Accuracy: 0.5202492211838006
